# Itnroduction and Setup

In [12]:
%pip install -q google-generativeai langchain langchain-community langchain-google-genai faiss-cpu python-dotenv gradio pandas numpy matplotlib beautifulsoup4 requests


[notice] A new release of pip is available: 23.2.1 -> 25.0.1
[notice] To update, run: pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [13]:
# Import necessary libraries
import os
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import google.generativeai as genai
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain.vectorstores import FAISS
from langchain.chains import ConversationalRetrievalChain
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory
from dotenv import load_dotenv
from IPython.display import display, Markdown, HTML

In [14]:
# Load environment variables (API keys)
load_dotenv()
# Configure Google Generative AI
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")                      
genai.configure(api_key=GOOGLE_API_KEY)

In [15]:
# Define models
generation_model = "gemini-1.5-flash"
embedding_model = "models/embedding-001"

# Initialize embeddings
embeddings = GoogleGenerativeAIEmbeddings(model=embedding_model)

# Initialize llm
llm = ChatGoogleGenerativeAI(model=generation_model)
llm.temperature = 0.7

In [16]:
def display_dnd(text, style="narrative"):
    """Display text with D&D-themed styling"""
    if style == "narrative":
        display(HTML(f"<div style='background-color: #f8f0e3; padding: 15px; border-radius: 10px; border: 1px solid #d0b894'>{text}</div>"))
    elif style == "rules":
        display(HTML(f"<div style='background-color: #e8f0f8; padding: 15px; border-radius: 10px; border: 1px solid #94b0d0'>{text}</div>"))
    elif style == "combat":
        display(HTML(f"<div style='background-color: #f8e3e3; padding: 15px; border-radius: 10px; border: 1px solid #d09494'>{text}</div>"))
    else:
        display(Markdown(text))

In [17]:
# Create project directories if they don't exist
os.makedirs("data", exist_ok=True)
os.makedirs("outputs", exist_ok=True)

In [18]:
import time

def rate_limited_query(question, max_retries=3, delay=60):
    for attempt in range(max_retries):
        try:
            return query_rules(question)
        except Exception as e:
            if "429" in str(e) and attempt < max_retries - 1:
                print(f"Rate limit hit. Waiting {delay} seconds...")
                time.sleep(delay)
            else:
                raise e

# D&D Rule Knowledge Base

In [19]:
import requests
from bs4 import BeautifulSoup
import re
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.docstore.document import Document

# Define basic SRD rules - in a real implementation, you would scrape or download these
# For this demo, we'll create a simplified set of core rules
basic_rules = """
        # Ability Scores
        There are six ability scores: Strength, Dexterity, Constitution, Intelligence, Wisdom, and Charisma.
        Ability modifiers are calculated as (score - 10) / 2, rounded down.
        # Combat
        Initiative is determined by rolling a d20 and adding your Dexterity modifier.
        On your turn, you can move up to your speed and take one action.
        Attack rolls are d20 + ability modifier + proficiency bonus (if proficient).
        Damage is determined by the weapon's damage die + ability modifier.
        # Saving Throws
        Saving throws are ability checks made to resist effects.
        Roll a d20 + ability modifier + proficiency bonus (if proficient).
        The DC is set by the effect's source.
        # Spellcasting
        Spell attack rolls are d20 + spellcasting ability modifier + proficiency bonus.
        Spell save DC = 8 + spellcasting ability modifier + proficiency bonus.
        Spell slots are consumed when casting spells and recovered during a long rest.
        # Resting
        Short Rest: At least 1 hour of rest. Can spend Hit Dice to recover hit points.
        Long Rest: At least 8 hours of rest. Recover all hit points and half of maximum Hit Dice.
        # Classes
        ## Fighter
        Hit Die: d10
        Primary Ability: Strength or Dexterity
        Saving Throw Proficiencies: Strength, Constitution
        Special Features: Second Wind, Action Surge, Extra Attack
        ## Wizard
        Hit Die: d6
        Primary Ability: Intelligence
        Saving Throw Proficiencies: Intelligence, Wisdom
        Special Features: Spellcasting, Arcane Recovery, Spell Mastery
        ## Cleric
        Hit Die: d8
        Primary Ability: Wisdom
        Saving Throw Proficiencies: Wisdom, Charisma
        Special Features: Spellcasting, Channel Divinity, Divine Intervention
        ## Rogue
        Hit Die: d8
        Primary Ability: Dexterity
        Saving Throw Proficiencies: Dexterity, Intelligence
        Special Features: Sneak Attack, Cunning Action, Evasion
        """
        # Additional rules for specific situations
advanced_rules = """
        # Conditions
        ## Blinded
        A blinded creature can't see and automatically fails any ability check that requires sight.
        Attack rolls against the creature have advantage, and the creature's attack rolls have disadvantage.
        ## Help
        ## Charmed
        A charmed creature can't attack the charmer or target them with harmful abilities or magical effects.
        The charmer has advantage on any ability check to interact socially with the creature.
        all_rules = basic_rules + "\n\n" + advanced_rules
        ## Exhaustion
        Exhaustion has six levels, each with increasing penalties:
        1. Disadvantage on ability checks
        2. Speed halved
        3. Disadvantage on attack rolls and saving throws
        4. Hit point maximum halved
        5. Speed reduced to 0
        6. Death
            chunk_size=500,
        # Combat Actions
        ## Dash
        When you take the Dash action, you gain extra movement for the current turn equal to your speed.
        rule_chunks = text_splitter.split_text(all_rules)
        ## Disengage
        If you take the Disengage action, your movement doesn't provoke opportunity attacks for the rest of the turn.
        ## Dodge
        When you take the Dodge action, until the start of your next turn, any attack roll made against you has disadvantage if you can see the attacker, and you make Dexterity saving throws with advantage.
        ## Help
        You can lend your aid to another creature in the completion of a task, giving them advantage on the next ability check they make for a specific task.
        Alternatively, you can aid a friendly creature in attacking a creature within 5 feet of you, giving the next attack roll advantage.
        """
# Combine all rules
all_rules = basic_rules + "\n\n" + advanced_rules
        
# Save rules to a file for persistence
with open("data/dnd_rules.txt", "w") as f:
    f.write(all_rules)
        
print(f"✅ Saved {len(all_rules)} characters of D&D rules to data/dnd_rules.txt")
        
# Process and chunk the rules
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    separators=["\n# ", "\n## ", "\n", " ", ""]
)
        
rule_chunks = text_splitter.split_text(all_rules)
rule_docs = [Document(page_content=chunk, metadata={"source": "dnd_rules"}) for chunk in rule_chunks]
        
print(f"📚 Created {len(rule_docs)} rule chunks for vector storage")

✅ Saved 3838 characters of D&D rules to data/dnd_rules.txt
📚 Created 10 rule chunks for vector storage


In [20]:
# Create a vector store from the rules
vectorstore = FAISS.from_documents(rule_docs, embeddings)
vectorstore.save_local("data/faiss_dnd_rules")
print("✅ Saved vectorstore to data/faiss_dnd_rules")

# Create a conversational retrieval chain
memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True,
)

# Define a custom prompt template
rules_prompt_template = """
You are a Dungeons and Dragons 5th Edition rules expert. Answer the user's questions based on the provided rules.
If you don't know the answer, provide your best guess based on the current rules set.

Rules Context:
{context}

Chat History:
{chat_history}

User Question: {question}

Your response (be concise, accurate, and helpful):

"""

PROMPT = PromptTemplate(
    input_variables=["context", "chat_history", "question"],
    template=rules_prompt_template,
)

rules_qa = ConversationalRetrievalChain.from_llm(
    llm = llm,
    retriever = vectorstore.as_retriever(search_kwargs={"k": 3}),
    memory = memory,
    combine_docs_chain_kwargs={"prompt": PROMPT},
)

print("✅ Created ConversationalRetrievalChain for D&D rules")

✅ Saved vectorstore to data/faiss_dnd_rules
✅ Created ConversationalRetrievalChain for D&D rules


In [21]:
# Function to query the rules knowledge base
def query_rules(question):
    """Query the D&D rules knowledge base"""
    response = rules_qa({"question": question})
    return response["answer"]

# Test the rules knowledge base with some exapmle questions
test_questions = [
    "What is the primary ability for a Fighter?",
    "How do you calculate a saving throw?",
    "What happens when you are blinded?",
    "What is the effect of the Help action?",
    "How do I calculate my character's ability modifier?",
    "What happens when a character is blinded?",
    "What are the special features of a Fighter?",
    "How does the Dodge action work in combat?"
]

print("🤖 Testing the D&D rules knowledge base with example questions...")
for question in test_questions:
    print(f"Q: {question}")
    answer = rate_limited_query(question)
    print(f"A: ")
    display_dnd(answer, style="rules")

🤖 Testing the D&D rules knowledge base with example questions...
Q: What is the primary ability for a Fighter?
A: 


Q: How do you calculate a saving throw?
A: 


Q: What happens when you are blinded?
A: 


Q: What is the effect of the Help action?
A: 


Q: How do I calculate my character's ability modifier?
A: 


Q: What happens when a character is blinded?
A: 


Q: What are the special features of a Fighter?
A: 


Q: How does the Dodge action work in combat?
A: 


# Cgaracter Management

In [22]:
# Define character schema and utilities
import json
import random
from typing import Dict, List, Optional, Union

# Define character schema as Python dataclass
CHARACTER_SCHEMA = {
    "name": "string",
    "race": "string",
    "class": "string",
    "level": "integer",
    "ability_scores": {
        "strength": "integer",
        "dexterity": "integer",
        "constitution": "integer",
        "intelligence": "integer",
        "wisdom": "integer",
        "charisma": "integer"
    },
    "hit_points": "integer",
    "armor_class": "integer",
    "proficiencies": ["string"],
    "equipment": ["string"],
    "background": "string",
    "traits": ["string"],
    "spells": ["string"]
}

# Function to calculate ability modifier
def calculate_modifier(score):
    return (score - 10) // 2

# Function to validate a character sheet against schema
def validate_character(character_data):
    """Validate character data against schema and D&D rules"""
    # Basic schema validation
    required_fields = ["name", "race", "class", "level", "ability_scores"]
    for field in required_fields:
        if field not in character_data:
            return False, f"Missing required field: {field}"
            
    # Ability score validation
    ability_scores = character_data.get("ability_scores", {})
    required_abilities = ["strength", "dexterity", "constitution", 
                         "intelligence", "wisdom", "charisma"]
    
    for ability in required_abilities:
        if ability not in ability_scores:
            return False, f"Missing ability score: {ability}"
        score = ability_scores[ability]
        if not isinstance(score, int) or score < 3 or score > 20:
            return False, f"Invalid {ability} score: {score}. Must be integer between 3-20."
    
    return True, "Character is valid"

# Function to display character sheet
def display_character_sheet(character):
    """Display a formatted character sheet"""
    if not character:
        print("No character data to display")
        return
        
    html = f"""
    <div style='background-color: #f0e6d2; padding: 20px; border-radius: 10px; border: 2px solid #c0a080; font-family: serif;'>
        <h2 style='text-align: center; border-bottom: 1px solid #c0a080;'>{character['name']}</h2>
        <p><b>Level {character['level']} {character['race']} {character['class']}</b><br>
        <i>{character.get('background', '')}</i></p>
        
        <div style='display: flex; justify-content: space-between;'>
            <div style='flex: 1;'>
                <h3>Ability Scores</h3>
                <ul>
    """
    
    abilities = character.get('ability_scores', {})
    for ability, score in abilities.items():
        modifier = calculate_modifier(score)
        sign = '+' if modifier >= 0 else ''
        html += f"<li><b>{ability.capitalize()}:</b> {score} ({sign}{modifier})</li>\n"
    
    html += f"""
                </ul>
            </div>
            <div style='flex: 1;'>
                <h3>Combat</h3>
                <ul>
                    <li><b>Hit Points:</b> {character.get('hit_points', 0)}</li>
                    <li><b>Armor Class:</b> {character.get('armor_class', 10)}</li>
                </ul>
                
                <h3>Proficiencies</h3>
                <ul>
    """
    
    for prof in character.get('proficiencies', []):
        html += f"<li>{prof}</li>\n"
    
    html += f"""
                </ul>
            </div>
            <div style='flex: 1;'>
                <h3>Equipment</h3>
                <ul>
    """
    
    for item in character.get('equipment', []):
        html += f"<li>{item}</li>\n"
    
    html += f"""
                </ul>
                
                <h3>Spells</h3>
                <ul>
    """
    
    for spell in character.get('spells', []):
        html += f"<li>{spell}</li>\n"
    
    html += """
                </ul>
            </div>
        </div>
        
        <h3>Character Traits</h3>
        <ul>
    """
    
    for trait in character.get('traits', []):
        html += f"<li>{trait}</li>\n"
    
    html += """
        </ul>
    </div>
    """
    
    display(HTML(html))

# Helper function to roll ability scores (4d6 drop lowest)
def roll_ability_score():
    rolls = [random.randint(1, 6) for _ in range(4)]
    return sum(sorted(rolls)[1:])  # Drop the lowest roll

In [23]:
# Character generator using structured output from Gemini
def generate_character(character_class=None, character_race=None, level=1, name=None):
    """Generate a D&D character using structured output from Gemini"""
    
    # Generate using a structured prompt for the LLM
    system_prompt = """
    You are an expert D&D character creator. Create a detailed D&D 5e character according to the specifications.
    Your response must be valid JSON that matches the following schema exactly:
    
    {
      "name": string,
      "race": string,
      "class": string,
      "level": integer,
      "ability_scores": {
        "strength": integer (3-20),
        "dexterity": integer (3-20),
        "constitution": integer (3-20),
        "intelligence": integer (3-20),
        "wisdom": integer (3-20),
        "charisma": integer (3-20)
      },
      "hit_points": integer,
      "armor_class": integer,
      "proficiencies": [strings],
      "equipment": [strings],
      "background": string,
      "traits": [strings],
      "spells": [strings]
    }
    
    Follow these D&D 5e rules precisely:
    1. Primary ability scores for classes: Fighter (STR/DEX), Wizard (INT), Cleric (WIS), Rogue (DEX)
    2. Generate realistic ability scores (higher in primary abilities)
    3. HP calculation: (Hit Die average + CON modifier) × level
    4. Include only equipment and proficiencies appropriate to the class
    5. Include spells only for spellcasting classes
    """
    
    # Create the user prompt based on specifications
    user_prompt = f"Generate a level {level} "
    if character_race:
        user_prompt += f"{character_race} "
    if character_class:
        user_prompt += f"{character_class} "
    user_prompt += "character"
    if name:
        user_prompt += f" named {name}"
    user_prompt += ". Make this character balanced but interesting."
    
    # Call the Gemini model with structured output
    try:
        response = genai.GenerativeModel(
            model_name=generation_model,
            generation_config={
                "temperature": 0.7,
                "response_mime_type": "application/json",
            },
        ).generate_content(
            [
                {"role": "system", "parts": [system_prompt]},
                {"role": "user", "parts": [user_prompt]}
            ]
        )
        
        character_json = json.loads(response.text)
        
        # Validate the character data
        valid, message = validate_character(character_json)
        if not valid:
            print(f"Warning: {message}")
            # Try to fix basic issues
            if "ability_scores" not in character_json:
                character_json["ability_scores"] = {
                    "strength": roll_ability_score(),
                    "dexterity": roll_ability_score(),
                    "constitution": roll_ability_score(),
                    "intelligence": roll_ability_score(),
                    "wisdom": roll_ability_score(),
                    "charisma": roll_ability_score()
                }
        
        # Save the character to a file
        character_filename = f"outputs/{character_json['name'].replace(' ', '_').lower()}.json"
        with open(character_filename, 'w') as f:
            json.dump(character_json, f, indent=2)
        print(f"✅ Character saved to {character_filename}")
        
        return character_json
    
    except Exception as e:
        print(f"Error generating character: {e}")
        # Fallback: generate a simple character with random stats
        print("Falling back to basic character generation...")
        return generate_basic_character(character_class, character_race, level, name)

def generate_basic_character(character_class=None, character_race=None, level=1, name=None):
    """Generate a basic character with random stats as fallback"""
    classes = ["Fighter", "Wizard", "Cleric", "Rogue"]
    races = ["Human", "Elf", "Dwarf", "Halfling"]
    
    char_class = character_class or random.choice(classes)
    char_race = character_race or random.choice(races)
    char_name = name or f"{char_race} {char_class}#{random.randint(1000, 9999)}"
    
    # Generate ability scores (4d6 drop lowest)
    ability_scores = {
        "strength": roll_ability_score(),
        "dexterity": roll_ability_score(),
        "constitution": roll_ability_score(),
        "intelligence": roll_ability_score(),
        "wisdom": roll_ability_score(),
        "charisma": roll_ability_score()
    }
    
    # Simple rules for hit points
    hit_die = {
        "Fighter": 10,
        "Wizard": 6,
        "Cleric": 8,
        "Rogue": 8
    }.get(char_class, 8)
    
    con_mod = calculate_modifier(ability_scores["constitution"])
    hit_points = hit_die + con_mod + (level - 1) * ((hit_die // 2 + 1) + con_mod)
    
    # Basic armor class
    armor_class = 10 + calculate_modifier(ability_scores["dexterity"])
    
    return {
        "name": char_name,
        "race": char_race,
        "class": char_class,
        "level": level,
        "ability_scores": ability_scores,
        "hit_points": hit_points,
        "armor_class": armor_class,
        "proficiencies": ["Simple Weapons"],
        "equipment": ["Adventurer's Pack", "Dagger"],
        "background": "Adventurer",
        "traits": ["Brave", "Cautious"],
        "spells": []
    }

In [24]:
# Test character generation with a few examples
print("🧙‍♂️ Generating D&D characters...\n")

# Generate a fighter
fighter = generate_character(character_class="Fighter", character_race="Dwarf", level=3, name="Thorin Stoneshield")
display_character_sheet(fighter)

# Generate a wizard
wizard = generate_character(character_class="Wizard", character_race="High Elf", level=2)
display_character_sheet(wizard)

# Generate a random character
random_character = generate_character(level=1)
display_character_sheet(random_character)

🧙‍♂️ Generating D&D characters...

Error generating character: 400 Content with system role is not supported.
Falling back to basic character generation...


Error generating character: 400 Content with system role is not supported.
Falling back to basic character generation...


Error generating character: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 2
}
]
Falling back to basic character generation...


In [25]:
# Function to modify a character (level up, add equipment, etc.)
def modify_character(character, modifications):
    """Apply modifications to a character"""
    if not character:
        return None
        
    # Create a copy to avoid modifying the original
    updated_character = character.copy()
    
    # Apply modifications
    for key, value in modifications.items():
        if key == "level_up":
            # Handle level up
            if value == True:
                updated_character["level"] = updated_character.get("level", 1) + 1
                
                # Update hit points
                hit_die = {
                    "Fighter": 10,
                    "Wizard": 6,
                    "Cleric": 8,
                    "Rogue": 8
                }.get(updated_character.get("class"), 8)
                
                con_mod = calculate_modifier(updated_character["ability_scores"]["constitution"])
                hp_gain = max(1, (hit_die // 2 + 1) + con_mod)
                updated_character["hit_points"] = updated_character.get("hit_points", 0) + hp_gain
                
        elif key == "add_equipment":
            # Add new equipment
            if isinstance(value, list):
                equipment = updated_character.get("equipment", [])
                equipment.extend(value)
                updated_character["equipment"] = equipment
                
        elif key == "add_spells":
            # Add new spells
            if isinstance(value, list):
                spells = updated_character.get("spells", [])
                spells.extend(value)
                updated_character["spells"] = spells
                
        elif key == "ability_score_increase":
            # Handle ability score increases
            if isinstance(value, dict):
                for ability, increase in value.items():
                    if ability in updated_character["ability_scores"]:
                        current = updated_character["ability_scores"][ability]
                        updated_character["ability_scores"][ability] = min(20, current + increase)
        else:
            # Handle direct replacements
            updated_character[key] = value
    
    # Validate the updated character
    valid, message = validate_character(updated_character)
    if not valid:
        print(f"Warning: {message}")
        return character  # Return original if update is invalid
    
    return updated_character

# Function to generate a party of characters
def generate_party(size=4, level=1):
    """Generate a balanced party of D&D characters"""
    # Ensure a balanced party with common roles
    classes = ["Fighter", "Wizard", "Cleric", "Rogue"]
    races = ["Human", "Elf", "Dwarf", "Halfling", "Half-Orc", "Gnome", "Tiefling"]
    
    party = []
    
    # Ensure we have the core classes if party size allows
    for i in range(min(size, len(classes))):
        char_class = classes[i]
        char_race = random.choice(races)
        character = generate_character(character_class=char_class, character_race=char_race, level=level)
        party.append(character)
    
    # Add any additional characters with random classes
    for i in range(len(classes), size):
        char_class = random.choice(classes)
        char_race = random.choice(races)
        character = generate_character(character_class=char_class, character_race=char_race, level=level)
        party.append(character)
    
    return party

In [26]:
# Test character modification
print("✨ Testing character modification (level up)\n")

# Level up the fighter
if fighter:
    print("Original character:")
    display_character_sheet(fighter)
    
    # Apply modifications
    modifications = {
        "level_up": True,
        "add_equipment": ["Potion of Healing", "Silver Warhammer"],
        "ability_score_increase": {"strength": 1}
    }
    
    updated_fighter = modify_character(fighter, modifications)
    
    print("\nAfter level up and modifications:")
    display_character_sheet(updated_fighter)
    
    # Save the updated character
    character_filename = f"outputs/{updated_fighter['name'].replace(' ', '_').lower()}_updated.json"
    with open(character_filename, 'w') as f:
        json.dump(updated_fighter, f, indent=2)
    print(f"✅ Updated character saved to {character_filename}")

✨ Testing character modification (level up)

Original character:



After level up and modifications:


✅ Updated character saved to outputs/thorin_stoneshield_updated.json


In [27]:
# Generate a balanced party for an adventure
print("🧝‍♀️🧙‍♂️👹🏹 Generating an adventuring party...\n")

party = generate_party(size=4, level=2)

for i, character in enumerate(party, 1):
    print(f"Party Member #{i}: {character['name']} - Level {character['level']} {character['race']} {character['class']}")
    display_character_sheet(character)

🧝‍♀️🧙‍♂️👹🏹 Generating an adventuring party...

Error generating character: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 2
}
]
Falling back to basic character generation...
Error generating character: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 2
}
]
Falling back to basic character generation...
Error generating character: 429 You exceeded your current quota, please check your plan and billing details

Party Member #2: Half-Orc Wizard#7338 - Level 2 Half-Orc Wizard


Party Member #3: Half-Orc Cleric#6563 - Level 2 Half-Orc Cleric


Party Member #4: Elf Rogue#4881 - Level 2 Elf Rogue


# Narrative Generation

In [28]:
# Narrative Generation System
import re
from typing import List, Dict, Any, Optional

# Define narrative types
NARRATIVE_TYPES = {
    "scene_description": "Vivid description of a location or setting",
    "npc_interaction": "Dialogue and interactions with non-player characters",
    "combat_narration": "Dramatic narration of combat encounters",
    "quest_hook": "Intriguing quest or mission introduction",
    "plot_twist": "Unexpected story development",
    "travel_montage": "Description of the party's journey",
    "flashback": "Character backstory or historical event"
}

# Example templates to use as few-shot learning examples
NARRATIVE_EXAMPLES = {
    "scene_description": [
        {
            "setting": "Ancient forest",
            "mood": "mysterious",
            "time": "dusk",
            "narrative": "The ancient trees of the Whispering Woods tower above you, their gnarled branches weaving together to form a verdant canopy that filters the waning daylight. As dusk settles, motes of luminescent pollen drift through the air like fireflies, casting an ethereal blue glow across the moss-covered stones that mark your path. A distant howl echoes through the trees, sending a shiver down your spine. The forest feels alive, watching, waiting."
        },
        {
            "setting": "Mountain village",
            "mood": "peaceful",
            "time": "morning",
            "narrative": "Dawn breaks over the village of Highcrest, painting the thatched roofs with golden light. Smoke rises from stone chimneys as villagers begin their day, the scent of fresh bread mixing with mountain air. Goats bleat from small pens, and children chase each other through narrow paths between the weathered buildings. The imposing peaks of the Frostspine Mountains loom protectively over the settlement, their snow-capped summits gleaming in the morning sun."
        }
    ],
    "npc_interaction": [
        {
            "npc_name": "Grimble Thorne",
            "npc_role": "Tavern keeper",
            "personality": "Gruff but kind-hearted",
            "situation": "Party seeking information",
            "narrative": "Grimble wipes the wooden counter with a rag that's seen better days, his bushy eyebrows furrowing as you mention the strange symbols found in the caves.\n\n\"Ain't heard nothing about no symbols,\" he grumbles, pouring you another ale without being asked. \"But old Widow Marlen, she used to study such things before her eyes went. Lives in the cottage with the blue door, end of the lane.\" He leans forward, voice dropping to a whisper. \"Don't mention my name, though. Still owes me for three months' worth of mead.\""
        }
    ],
    "combat_narration": [
        {
            "enemies": "Goblin ambush",
            "environment": "Narrow ravine",
            "stakes": "Protecting a merchant caravan",
            "narrative": "The first arrow whistles past your ear as cackling goblins pour from the crevices above the ravine. The merchant's horses rear in panic, nearly overturning the wagon of precious silks. A goblin leaps down, yellow eyes gleaming with malice, crude blade aimed at the caravan master's throat—until your shield intercepts with a resonant clang that echoes through the ravine. Battle is joined, the clash of steel mingling with war cries and the terrified shouts of the merchants scrambling for cover behind their wagons."
        }
    ]
}

def generate_narrative(narrative_type: str, context: Dict[str, Any], party=None, style="descriptive", length="medium"):
    """
    Generate narrative content based on the specified type and context.
    
    Args:
        narrative_type: Type of narrative (scene, npc, combat, etc.)
        context: Dict containing contextual information for the narrative
        party: Optional party information to include characters in the narrative
        style: Narrative style (descriptive, dramatic, humorous, etc.)
        length: Desired length (short, medium, long)
    
    Returns:
        Generated narrative text
    """
    # Construct few-shot examples based on the narrative type
    few_shot_examples = ""
    if narrative_type in NARRATIVE_EXAMPLES:
        examples = NARRATIVE_EXAMPLES[narrative_type]
        for example in examples:
            few_shot_examples += f"Example {narrative_type} context: {example}\n\n"
            few_shot_examples += f"Generated narrative:\n{example['narrative']}\n\n---\n\n"
    
    # Create a prompt with context-specific elements
    system_prompt = f"""
    You are an expert Dungeons & Dragons Dungeon Master with a talent for vivid, engaging narrative descriptions.
    Create a {style} {narrative_type} for a D&D adventure with a {length} length.
    
    Make your narrative:
    - Evocative and sensory, engaging all five senses where appropriate
    - Balanced between description, action, and atmosphere
    - Written in second person perspective when describing what the players experience
    - Authentic to D&D's fantasy setting
    - Include hooks or openings for player interaction
    
    {few_shot_examples}
    """
    
    # Add party context if provided
    party_context = ""
    if party:
        party_names = [character.get('name', f"the {character.get('race', '')} {character.get('class', '')}") 
                      for character in party if character]
        party_context = f"The party consists of {', '.join(party_names)}. "
    
    # Create the specific request based on narrative type
    user_prompt = f"Create a {narrative_type} with the following details:\n"
    for key, value in context.items():
        user_prompt += f"- {key}: {value}\n"
    
    if party_context:
        user_prompt += f"\n{party_context}Include these characters in the narrative where appropriate."
    
    # Call the Gemini model
    try:
        response = genai.GenerativeModel(
            model_name=generation_model,
            generation_config={
                "temperature": 0.8,
                "top_p": 0.9,
                "max_output_tokens": 1000 if length == "long" else 500 if length == "medium" else 250
            }
        ).generate_content(
            [
                {"role": "system", "parts": [system_prompt]},
                {"role": "user", "parts": [user_prompt]}
            ]
        )
        
        narrative = response.text
        
        # Post-process to fix common narrative issues
        narrative = re.sub(r'^\s*["\']|["\']\s*$', '', narrative)  # Remove quotation marks at start/end
        
        return narrative
        
    except Exception as e:
        print(f"Error generating narrative: {e}")
        return f"*The Dungeon Master pauses, shuffling through notes...*\n\nAs you wait, your party contemplates what might happen next in your adventure."

# Function to create a complete adventure premise
def generate_adventure_premise(theme=None, level_range=None, location=None, villain_type=None):
    """Generate a complete adventure premise with hooks, NPCs, and plot points"""
    
    context = {
        "theme": theme or "Choose a compelling theme",
        "level_range": level_range or "appropriate for the party level",
        "location": location or "Choose an interesting location",
        "villain_type": villain_type or "Choose a suitable antagonist"
    }
    
    system_prompt = """
    You are an expert D&D adventure designer. Create a complete adventure premise that would be suitable
    for a 3-5 session mini-campaign. 
    
    Your response should be structured as follows:
    
    # [COMPELLING ADVENTURE TITLE]
    
    ## Overview
    [1-2 paragraph summary of the adventure]
    
    ## Hook
    [How the players get involved]
    
    ## Key NPCs
    - [NPC name]: [Brief description and role]
    - [NPC name]: [Brief description and role]
    - [NPC name]: [Brief description and role]
    
    ## Key Locations
    - [Location name]: [Brief description]
    - [Location name]: [Brief description]
    - [Location name]: [Brief description]
    
    ## Main Plot Points
    1. [First major event/encounter]
    2. [Second major event/encounter]
    3. [Third major event/encounter]
    4. [Climax]
    
    ## Potential Side Quests
    - [Side quest idea]
    - [Side quest idea]
    
    ## Rewards
    [Potential treasures, items, or other rewards]
    """
    
    user_prompt = f"""
    Create a D&D adventure premise with these parameters:
    - Theme/Type: {context['theme']}
    - Level Range: {context['level_range']}
    - Main Location: {context['location']}
    - Villain Type: {context['villain_type']}
    
    Make this adventure flexible enough to adapt to different player choices while maintaining an engaging story arc.
    """
    
    try:
        response = genai.GenerativeModel(
            model_name=generation_model,
            generation_config={
                "temperature": 0.7,
                "top_p": 0.9,
            }
        ).generate_content(
            [
                {"role": "system", "parts": [system_prompt]},
                {"role": "user", "parts": [user_prompt]}
            ]
        )
        
        return response.text
        
    except Exception as e:
        print(f"Error generating adventure premise: {e}")
        return "Could not generate adventure premise due to an error."

# Function to create interactive narration that responds to player choices
def create_narrative_with_choices(scene_context, choices=None):
    """Generate narrative with potential outcomes for each player choice"""
    
    if choices is None:
        choices = ["Investigate further", "Talk to nearby NPCs", "Leave the area"]
    
    system_prompt = """
    You are an expert D&D Dungeon Master. Create a narrative scene with multiple possible outcomes
    based on different player choices. For each choice, provide a brief description of what happens
    if players select that option. Make each outcome interesting and lead to further adventure opportunities.
    
    Format your response as:
    
    [Scene description]
    
    ## Possible Actions:
    1. [Choice 1]
    2. [Choice 2]
    3. [Choice 3]
    
    ## Outcomes:
    1. If the players [Choice 1]: [Outcome description]
    2. If the players [Choice 2]: [Outcome description]
    3. If the players [Choice 3]: [Outcome description]
    """
    
    choices_text = "\n".join([f"- {choice}" for choice in choices])
    
    user_prompt = f"""
    Create a narrative scene with outcomes for these player choices:
    
    Scene context: {scene_context}
    
    Player choices:
    {choices_text}
    
    Provide an engaging scene description and interesting outcomes for each choice.
    """
    
    try:
        response = genai.GenerativeModel(
            model_name=generation_model,
            generation_config={
                "temperature": 0.7,
            }
        ).generate_content(
            [
                {"role": "system", "parts": [system_prompt]},
                {"role": "user", "parts": [user_prompt]}
            ]
        )
        
        return response.text
        
    except Exception as e:
        print(f"Error generating narrative with choices: {e}")
        return f"*The Dungeon Master ponders the possible outcomes...*"

In [29]:
print("🧙‍♂️ Testing Narrative Generation\n")

# 1. Generate a scene description
print("Generating a scene description...")
scene_context = {
    "setting": "Abandoned temple",
    "mood": "foreboding",
    "time": "midnight",
    "special_element": "A mysterious altar glowing with arcane runes"
}
scene_description = generate_narrative("scene_description", scene_context)
display_dnd(scene_description, style="narrative")
print("\n")

# 2. Generate an NPC interaction
print("Generating an NPC interaction...")
npc_context = {
    "npc_name": "Lady Ravenna Nightshade",
    "npc_role": "Noble vampire",
    "personality": "Elegant, manipulative, centuries old",
    "situation": "Offering the party a deal they shouldn't refuse",
    "hidden_agenda": "Seeks an ancient artifact the party recently acquired"
}
npc_dialogue = generate_narrative("npc_interaction", npc_context)
display_dnd(npc_dialogue, style="narrative")
print("\n")

# 3. Generate a combat narration
print("Generating a combat narration...")
combat_context = {
    "enemies": "A young red dragon",
    "environment": "Volcanic cavern with pools of lava",
    "stakes": "Protecting a village in the valley below",
    "dramatic_moment": "The dragon reveals it can control the volcano"
}
combat_narrative = generate_narrative("combat_narration", combat_context)
display_dnd(combat_narrative, style="combat")
print("\n")

# 4. Generate an adventure premise
print("Generating a complete adventure premise...")
adventure = generate_adventure_premise(
    theme="A curse spreading through a royal bloodline",
    level_range="5-8",
    location="A kingdom on the edge of a haunted forest",
    villain_type="Ancient witch seeking revenge"
)
display_dnd(adventure)
print("\n")

# 5. Generate narrative with player choices
print("Generating narrative with player choices...")
choice_narrative = create_narrative_with_choices(
    "The party discovers a hidden door beneath the tavern, with strange sounds coming from beyond it",
    choices=[
        "Break down the door immediately",
        "Listen carefully for clues",
        "Ask the tavern keeper about the door",
        "Cast detect magic on the door"
    ]
)
display_dnd(choice_narrative)

🧙‍♂️ Testing Narrative Generation

Generating a scene description...
Error generating narrative: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 1
}
]




Generating an NPC interaction...
Error generating narrative: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 1
}
]




Generating a combat narration...
Error generating narrative: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 1
}
]




Generating a complete adventure premise...
Error generating adventure premise: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 1
}
]




Generating narrative with player choices...
Error generating narrative with choices: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 1
}
]


In [30]:
print("🔍 Generating a narrative featuring the party's characters...\n")

# Test narrative generation with actual party members (if available)
try:
    if 'party' in globals() and party:
        party_scene_context = {
            "setting": "Ancient dwarven forge",
            "situation": "The discovery of a legendary weapon",
            "complication": "The forge's guardian awakens",
            "atmosphere": "Awe-inspiring but dangerous"
        }
        
        party_narrative = generate_narrative(
            "scene_description", 
            party_scene_context, 
            party=party,
            style="epic",
            length="medium"
        )
        
        print("A scene featuring your adventuring party:")
        display_dnd(party_narrative, style="narrative")
    else:
        # Create an example with generic characters
        sample_party = [
            {"name": "Thorgrim", "race": "Dwarf", "class": "Fighter"},
            {"name": "Elyndra", "race": "Elf", "class": "Wizard"},
            {"name": "Brother Dorian", "race": "Human", "class": "Cleric"},
            {"name": "Nimble", "race": "Halfling", "class": "Rogue"}
        ]
        
        party_scene_context = {
            "setting": "Ancient dragon's lair",
            "situation": "Finding the treasure hoard",
            "complication": "The dragon is only pretending to be asleep",
            "atmosphere": "Tense and dangerous"
        }
        
        party_narrative = generate_narrative(
            "scene_description", 
            party_scene_context, 
            party=sample_party,
            style="suspenseful",
            length="medium"
        )
        
        print("An example scene with a sample adventuring party:")
        display_dnd(party_narrative, style="narrative")
except Exception as e:
    print(f"Error generating party narrative: {e}")

🔍 Generating a narrative featuring the party's characters...

Error generating narrative: 429 You exceeded your current quota, please check your plan and billing details. For more information on this error, head to: https://ai.google.dev/gemini-api/docs/rate-limits. [violations {
}
, links {
  description: "Learn more about Gemini API quotas"
  url: "https://ai.google.dev/gemini-api/docs/rate-limits"
}
, retry_delay {
  seconds: 1
}
]
A scene featuring your adventuring party:


# Encounter Builder

In [31]:
import random
import math
from typing import List, Dict, Any, Optional

# Monster difficulty ratings and XP values
CR_TO_XP = {
    0: 10, 0.125: 25, 0.25: 50, 0.5: 100, 1: 200, 2: 450, 3: 700, 4: 1100, 
    5: 1800, 6: 2300, 7: 2900, 8: 3900, 9: 5000, 10: 5900, 11: 7200, 12: 8400,
    13: 10000, 14: 11500, 15: 13000, 16: 15000, 17: 18000, 18: 20000, 19: 22000, 20: 25000
}

# Encounter difficulty XP multipliers
DIFFICULTY_MULTIPLIERS = {
    "easy": 1,
    "medium": 2,
    "hard": 3,
    "deadly": 4
}

# XP thresholds by character level
XP_THRESHOLDS = {
    # Level: [Easy, Medium, Hard, Deadly]
    1: [25, 50, 75, 100],
    2: [50, 100, 150, 200],
    3: [75, 150, 225, 400],
    4: [125, 250, 375, 500],
    5: [250, 500, 750, 1100],
    6: [300, 600, 900, 1400],
    7: [350, 750, 1100, 1700],
    8: [450, 900, 1400, 2100],
    9: [550, 1100, 1600, 2400],
    10: [600, 1200, 1900, 2800],
    11: [800, 1600, 2400, 3600],
    12: [1000, 2000, 3000, 4500],
    13: [1100, 2200, 3400, 5100],
    14: [1250, 2500, 3800, 5700],
    15: [1400, 2800, 4300, 6400],
    16: [1600, 3200, 4800, 7200],
    17: [2000, 3900, 5900, 8800],
    18: [2100, 4200, 6300, 9500],
    19: [2400, 4900, 7300, 10900],
    20: [2800, 5700, 8500, 12700]
}

# Basic monster database
MONSTER_DATABASE = {
    "Goblin": {"cr": 0.25, "type": "humanoid", "environment": ["forest", "cave", "mountain"], "traits": ["nimble escape"]},
    "Wolf": {"cr": 0.25, "type": "beast", "environment": ["forest", "mountain", "tundra"], "traits": ["pack tactics"]},
    "Orc": {"cr": 0.5, "type": "humanoid", "environment": ["forest", "mountain", "underground"], "traits": ["aggressive"]},
    "Bandit": {"cr": 0.125, "type": "humanoid", "environment": ["urban", "forest", "road"], "traits": ["pack tactics"]},
    "Zombie": {"cr": 0.25, "type": "undead", "environment": ["dungeon", "graveyard", "swamp"], "traits": ["undead fortitude"]},
    "Skeleton": {"cr": 0.25, "type": "undead", "environment": ["dungeon", "graveyard", "ruins"], "traits": ["vulnerable to bludgeoning"]},
    "Giant Spider": {"cr": 1, "type": "beast", "environment": ["forest", "cave", "underground"], "traits": ["web", "spider climb"]},
    "Bugbear": {"cr": 1, "type": "humanoid", "environment": ["forest", "cave", "mountain"], "traits": ["brute", "surprise attack"]},
    "Animated Armor": {"cr": 1, "type": "construct", "environment": ["dungeon", "tower", "castle"], "traits": ["antimagic susceptibility"]},
    "Young Dragon": {"cr": 6, "type": "dragon", "environment": ["mountain", "cave", "ruins"], "traits": ["breath weapon", "frightful presence"]},
    "Troll": {"cr": 5, "type": "giant", "environment": ["forest", "mountain", "swamp"], "traits": ["regeneration"]},
    "Owlbear": {"cr": 3, "type": "monstrosity", "environment": ["forest", "mountain"], "traits": ["keen sight and smell"]},
    "Hill Giant": {"cr": 5, "type": "giant", "environment": ["hill", "mountain"], "traits": ["poor depth perception"]},
    "Mimic": {"cr": 2, "type": "monstrosity", "environment": ["dungeon", "cave", "ruins"], "traits": ["shapechanger", "adhesive"]},
    "Beholder": {"cr": 13, "type": "aberration", "environment": ["underground", "ruins", "lair"], "traits": ["antimagic cone", "eye rays"]}
}

def calculate_encounter_difficulty(party, monsters):
    """Calculate the difficulty of an encounter based on party composition and monsters"""
    if not party or not monsters:
        return "unknown", 0
    
    # Get party level and size
    party_size = len(party)
    party_levels = [char.get("level", 1) for char in party]
    avg_party_level = sum(party_levels) / len(party_levels)
    
    # Calculate XP thresholds for this party
    party_thresholds = {"easy": 0, "medium": 0, "hard": 0, "deadly": 0}
    
    for level in party_levels:
        thresholds = XP_THRESHOLDS.get(level, XP_THRESHOLDS[1])  # Default to level 1 if not found
        party_thresholds["easy"] += thresholds[0]
        party_thresholds["medium"] += thresholds[1]
        party_thresholds["hard"] += thresholds[2]
        party_thresholds["deadly"] += thresholds[3]
    
    # Calculate monster XP
    monster_count = len(monsters)
    total_monster_xp = sum([CR_TO_XP.get(monster.get("cr", 0), 0) for monster in monsters])
    
    # Apply encounter multiplier based on number of monsters
    if monster_count == 1:
        multiplier = 1
    elif monster_count == 2:
        multiplier = 1.5
    elif 3 <= monster_count <= 6:
        multiplier = 2
    elif 7 <= monster_count <= 10:
        multiplier = 2.5
    elif 11 <= monster_count <= 14:
        multiplier = 3
    else:
        multiplier = 4
    
    adjusted_xp = total_monster_xp * multiplier
    
    # Determine difficulty
    difficulty = "easy"
    if adjusted_xp >= party_thresholds["deadly"]:
        difficulty = "deadly"
    elif adjusted_xp >= party_thresholds["hard"]:
        difficulty = "hard"
    elif adjusted_xp >= party_thresholds["medium"]:
        difficulty = "medium"
    
    return difficulty, adjusted_xp

def generate_encounter(party=None, difficulty="medium", environment="dungeon", theme=None):
    """Generate a balanced encounter for the party"""
    # Default party if none provided
    if not party:
        party = [{"level": 3}, {"level": 3}, {"level": 3}, {"level": 3}]
    
    # Get party level and size
    party_size = len(party)
    party_levels = [char.get("level", 1) for char in party]
    avg_party_level = sum(party_levels) / len(party_levels)
    
    # Calculate target XP based on difficulty
    target_xp = 0
    for level in party_levels:
        thresholds = XP_THRESHOLDS.get(level, XP_THRESHOLDS[1])
        difficulty_index = ["easy", "medium", "hard", "deadly"].index(difficulty)
        target_xp += thresholds[difficulty_index]
    
    # Filter monsters by environment
    suitable_monsters = [
        (name, data) for name, data in MONSTER_DATABASE.items() 
        if environment in data.get("environment", [])
    ]
    
    if not suitable_monsters:
        suitable_monsters = list(MONSTER_DATABASE.items())
    
    # Generate encounter
    selected_monsters = []
    current_xp = 0
    max_iterations = 100  # Prevent infinite loops
    
    for _ in range(max_iterations):
        if current_xp >= target_xp * 0.8 and current_xp <= target_xp * 1.2:
            break
            
        # If we're over the target, remove a monster
        if current_xp > target_xp * 1.2 and selected_monsters:
            removed = selected_monsters.pop()
            current_xp -= CR_TO_XP.get(removed.get("cr", 0), 0)
            continue
            
        # Add a monster
        monster_name, monster_data = random.choice(suitable_monsters)
        monster_cr = monster_data.get("cr", 0)
        monster_xp = CR_TO_XP.get(monster_cr, 0)
        
        # Skip very powerful monsters for low-level parties
        if monster_cr > avg_party_level * 2 and avg_party_level < 5:
            continue
            
        # Create the monster entry
        monster = {
            "name": monster_name,
            "cr": monster_cr,
            "type": monster_data.get("type", "unknown"),
            "traits": monster_data.get("traits", []),
            "xp": monster_xp
        }
        
        selected_monsters.append(monster)
        current_xp += monster_xp
    
    # Calculate final difficulty
    final_difficulty, adjusted_xp = calculate_encounter_difficulty(party, selected_monsters)
    
    # Group monsters by type for cleaner output
    grouped_monsters = {}
    for monster in selected_monsters:
        name = monster["name"]
        if name in grouped_monsters:
            grouped_monsters[name]["count"] += 1
        else:
            grouped_monsters[name] = {
                "count": 1,
                "cr": monster["cr"],
                "type": monster["type"],
                "traits": monster["traits"]
            }
    
    # Create encounter data
    encounter = {
        "monsters": grouped_monsters,
        "total_monsters": len(selected_monsters),
        "difficulty": final_difficulty,
        "target_difficulty": difficulty,
        "adjusted_xp": adjusted_xp,
        "environment": environment,
        "theme": theme
    }
    
    return encounter

def describe_encounter(encounter, party=None):
    """Generate a description of the encounter"""
    # Create a prompt for the narrative generation
    environment = encounter.get("environment", "dungeon")
    monsters = encounter.get("monsters", {})
    difficulty = encounter.get("difficulty", "medium")
    
    # Basic environment descriptions
    environment_desc = {
        "forest": "a dense woodland with towering trees and dappled sunlight",
        "cave": "a dark, damp cavern with echoing sounds and limited visibility",
        "mountain": "a rocky mountainside with steep cliffs and thin air",
        "dungeon": "a dimly lit dungeon corridor with ancient stonework",
        "ruins": "crumbling ruins of a once-great structure, now reclaimed by nature",
        "swamp": "a fetid swamp with bubbling pools and twisted vegetation",
        "urban": "the shadowy backstreets of a settlement",
        "desert": "a harsh desert landscape with shifting sands and scorching heat",
        "underwater": "the mysterious depths of an underwater location",
        "plains": "open grasslands with little cover but excellent visibility",
        "tundra": "a frozen wasteland of ice and snow"
    }
    
    # Create context for narrative generation
    monster_list = []
    for name, data in monsters.items():
        count = data.get("count", 1)
        if count == 1:
            monster_list.append(f"a {name}")
        else:
            monster_list.append(f"{count} {name}s")
    
    monster_text = ", ".join(monster_list[:-1])
    if len(monster_list) > 1:
        monster_text += f" and {monster_list[-1]}"
    else:
        monster_text = monster_list[0]
    
    context = {
        "environment": environment_desc.get(environment, environment),
        "monsters": monster_text,
        "difficulty": difficulty,
        "surprise": random.choice(["The party spots the enemies first", 
                                  "The monsters are aware of the party's approach",
                                  "Both sides are surprised to encounter each other"])
    }
    
    # If we have the party, include them
    try:
        encounter_narrative = generate_narrative("combat_narration", context, party=party, style="dramatic")
        return encounter_narrative
    except:
        # Fallback if narrative generation fails
        return f"""
        As your party traverses {context['environment']}, you encounter {context['monsters']}. 
        {context['surprise']}. The battle looks to be a {difficulty} fight. 
        Prepare yourselves, for combat is imminent!
        """

def generate_encounter_with_narrative(party=None, difficulty="medium", environment="dungeon", theme=None):
    """Generate an encounter and its narrative description"""
    # Generate the encounter
    encounter = generate_encounter(party, difficulty, environment, theme)
    
    # Generate description
    narrative = describe_encounter(encounter, party)
    
    # Add encounter details to result
    result = {
        "encounter": encounter,
        "narrative": narrative
    }
    
    return result

def display_encounter(encounter_data):
    """Display an encounter in a formatted way"""
    encounter = encounter_data.get("encounter", {})
    narrative = encounter_data.get("narrative", "")
    
    # Display the narrative
    display_dnd(narrative, style="combat")
    
    # Display encounter details
    print("\n## Encounter Details:")
    print(f"Difficulty: {encounter.get('difficulty', 'unknown')} (Target: {encounter.get('target_difficulty', 'medium')})")
    print(f"Adjusted XP: {encounter.get('adjusted_xp', 0)} XP")
    print(f"Environment: {encounter.get('environment', 'unknown')}")
    if encounter.get("theme"):
        print(f"Theme: {encounter.get('theme')}")
    
    print("\n## Monsters:")
    for name, data in encounter.get("monsters", {}).items():
        count = data.get("count", 1)
        cr = data.get("cr", 0)
        monster_type = data.get("type", "unknown")
        traits = ", ".join(data.get("traits", []))
        
        print(f"- {name} (x{count}): CR {cr}, {monster_type.capitalize()}")
        if traits:
            print(f"  Special traits: {traits}")
    
    print("\nTotal monsters:", encounter.get("total_monsters", 0))

In [33]:
print("⚔️ Testing Encounter Builder\n")

# Generate an encounter for the party we created earlier
if 'party' in globals() and party:
    print("Generating an encounter for your adventuring party...")
    
    # Generate a dungeon encounter
    dungeon_encounter = generate_encounter_with_narrative(
        party=party, 
        difficulty="medium", 
        environment="dungeon",
        theme="Ancient dwarven ruins overrun by undead"
    )
    
    # Display it
    display_encounter(dungeon_encounter)
    
    # Generate a wilderness encounter
    print("\n\nGenerating a wilderness encounter...")
    wilderness_encounter = generate_encounter_with_narrative(
        party=party, 
        difficulty="hard", 
        environment="forest",
        theme="Goblinoid war party"
    )
    
    # Display it
    display_encounter(wilderness_encounter)
    
else:
    # Create a sample party if none exists
    sample_party = [
        {"name": "Thorgrim", "race": "Dwarf", "class": "Fighter", "level": 3},
        {"name": "Elyndra", "race": "Elf", "class": "Wizard", "level": 3},
        {"name": "Brother Dorian", "race": "Human", "class": "Cleric", "level": 3},
        {"name": "Nimble", "race": "Halfling", "class": "Rogue", "level": 3}
    ]
    
    print("Generating encounters for a sample party...")
    
    # Generate encounters at different difficulty levels
    for difficulty in ["easy", "medium", "hard", "deadly"]:
        print(f"\n\nGenerating a {difficulty} encounter...")
        encounter = generate_encounter_with_narrative(
            party=sample_party, 
            difficulty=difficulty, 
            environment=random.choice(["dungeon", "forest", "mountain", "ruins"])
        )
        
        # Display it
        display_encounter(encounter)

⚔️ Testing Encounter Builder

Generating an encounter for your adventuring party...
Error generating narrative: 400 Content with system role is not supported.



## Encounter Details:
Difficulty: hard (Target: medium)
Adjusted XP: 700 XP
Environment: dungeon
Theme: Ancient dwarven ruins overrun by undead

## Monsters:
- Animated Armor (x1): CR 1, Construct
  Special traits: antimagic susceptibility
- Zombie (x2): CR 0.25, Undead
  Special traits: undead fortitude
- Skeleton (x1): CR 0.25, Undead
  Special traits: vulnerable to bludgeoning

Total monsters: 4


Generating a wilderness encounter...
Error generating narrative: 400 Content with system role is not supported.



## Encounter Details:
Difficulty: deadly (Target: hard)
Adjusted XP: 1300 XP
Environment: forest
Theme: Goblinoid war party

## Monsters:
- Giant Spider (x3): CR 1, Beast
  Special traits: web, spider climb
- Wolf (x1): CR 0.25, Beast
  Special traits: pack tactics

Total monsters: 4


# Interactive DM Session

In [34]:
import gradio as gr
import os
import json
import random
from IPython.display import display, HTML

# Load saved party if exists
def load_saved_party():
    """Load previously saved party from file if it exists"""
    try:
        if os.path.exists("data/saved_party.json"):
            with open("data/saved_party.json", "r") as f:
                return json.load(f)
        return []
    except Exception as e:
        print(f"Error loading saved party: {e}")
        return []

# Save current party
def save_party(party):
    """Save party to file for persistence"""
    try:
        with open("data/saved_party.json", "w") as f:
            json.dump(party, f, indent=2)
        return True
    except Exception as e:
        print(f"Error saving party: {e}")
        return False

# Campaign state management
class CampaignState:
    def __init__(self):
        self.party = load_saved_party() or []
        self.current_location = "Tavern in the starting village"
        self.quest_log = []
        self.npcs_met = {}
        self.current_scene = "Your adventure begins in a cozy tavern. The fire crackles in the hearth as patrons chat quietly around you."
        self.game_history = ["Campaign started. The party meets in a tavern."]
        self.save()
    
    def add_to_history(self, event):
        """Add an event to the game history"""
        self.game_history.append(event)
        if len(self.game_history) > 50:  # Keep history manageable
            self.game_history = self.game_history[-50:]
    
    def update_location(self, new_location):
        """Update the party's current location"""
        self.add_to_history(f"Party traveled to {new_location}")
        self.current_location = new_location
        self.save()
    
    def add_quest(self, quest_title, quest_description):
        """Add a new quest to the quest log"""
        quest = {
            "title": quest_title,
            "description": quest_description,
            "status": "active"
        }
        self.quest_log.append(quest)
        self.add_to_history(f"New quest accepted: {quest_title}")
        self.save()
    
    def complete_quest(self, quest_index):
        """Mark a quest as completed"""
        if 0 <= quest_index < len(self.quest_log):
            self.quest_log[quest_index]["status"] = "completed"
            self.add_to_history(f"Quest completed: {self.quest_log[quest_index]['title']}")
            self.save()
    
    def add_npc(self, npc_name, npc_description):
        """Add an NPC to the campaign"""
        self.npcs_met[npc_name] = {
            "description": npc_description,
            "relationship": "neutral"
        }
        self.save()
    
    def set_scene(self, scene_description):
        """Set the current scene description"""
        self.current_scene = scene_description
        self.save()
    
    def save(self):
        """Save campaign state"""
        state = {
            "party": self.party,
            "current_location": self.current_location,
            "quest_log": self.quest_log,
            "npcs_met": self.npcs_met,
            "current_scene": self.current_scene,
            "game_history": self.game_history
        }
        try:
            with open("data/campaign_state.json", "w") as f:
                json.dump(state, f, indent=2)
        except Exception as e:
            print(f"Error saving campaign state: {e}")

    def load(self):
        """Load campaign state"""
        try:
            if os.path.exists("data/campaign_state.json"):
                with open("data/campaign_state.json", "r") as f:
                    state = json.load(f)
                    self.party = state.get("party", [])
                    self.current_location = state.get("current_location", "Tavern")
                    self.quest_log = state.get("quest_log", [])
                    self.npcs_met = state.get("npcs_met", {})
                    self.current_scene = state.get("current_scene", "")
                    self.game_history = state.get("game_history", ["Campaign loaded."])
                return True
            return False
        except Exception as e:
            print(f"Error loading campaign state: {e}")
            return False

# Initialize campaign state
campaign = CampaignState()
loaded = campaign.load()
if not loaded:
    print("Starting a new campaign...")
else:
    print("Loaded existing campaign.")

# Interactive DM system that ties all components together
def dm_session(user_input, game_state, session_history):
    """Process user input and generate DM response"""
    
    # Add user input to history
    if user_input:
        session_history += f"\nPlayer: {user_input}"
    
    # Initialize context for the AI DM
    system_prompt = """
    You are a Dungeons & Dragons Dungeon Master managing an interactive campaign.
    Use the provided game state to generate appropriate responses.
    Respond with evocative, descriptive language as a good DM would.
    You can reference characters, locations, quests, and NPCs from the game state.
    When appropriate, suggest actions or choices for the players.
    
    Your response should ONLY be your in-character DM narration. Do not include meta-commentary.
    """
    
    # Handle special commands
    if user_input.startswith("/"):
        command_parts = user_input[1:].split(" ", 1)
        command = command_parts[0].lower()
        
        if command == "help":
            return session_history + "\n\nDM: Available commands:\n/help - Show this help message\n/roll XdY - Roll X dice with Y sides\n/check [ability] - Make an ability check\n/encounter [difficulty] - Generate a random encounter\n/scene [setting] - Generate a scene description\n/npc [name] - Generate an NPC interaction\n/quest - Generate a new quest\n/rules [query] - Look up a rule", game_state
        
        elif command == "roll":
            # Handle dice rolling
            try:
                if len(command_parts) > 1:
                    dice_spec = command_parts[1]
                    parts = dice_spec.lower().split('d')
                    if len(parts) == 2:
                        num_dice = int(parts[0]) if parts[0] else 1
                        dice_sides = int(parts[1])
                        
                        if num_dice > 100 or dice_sides > 1000:  # Sanity limits
                            return session_history + "\n\nDM: That's too many dice or sides! Please keep it reasonable.", game_state
                        
                        rolls = [random.randint(1, dice_sides) for _ in range(num_dice)]
                        total = sum(rolls)
                        result = f"Rolling {num_dice}d{dice_sides}: {rolls} = {total}"
                        return session_history + f"\n\nDM: {result}", game_state
            except:
                pass
            
            return session_history + "\n\nDM: Invalid roll format. Use '/roll XdY' where X is the number of dice and Y is the sides (e.g., /roll 2d6).", game_state
        
        elif command == "rules":
            # Look up a rule
            if len(command_parts) > 1:
                rule_query = command_parts[1]
                try:
                    rule_answer = query_rules(rule_query)
                    return session_history + f"\n\nDM: Rule lookup for '{rule_query}':\n{rule_answer}", game_state
                except Exception as e:
                    return session_history + f"\n\nDM: Sorry, I couldn't look up that rule. Error: {str(e)}", game_state
            else:
                return session_history + "\n\nDM: Please specify what rule you're looking for. For example: /rules how does combat work?", game_state
        
        elif command == "encounter":
            # Generate an encounter
            difficulty = "medium"
            if len(command_parts) > 1:
                difficulty = command_parts[1].lower()
                if difficulty not in ["easy", "medium", "hard", "deadly"]:
                    difficulty = "medium"
            
            try:
                # Get the party from game state
                party_data = json.loads(game_state).get("party", [])
                if not party_data:
                    return session_history + "\n\nDM: You need characters in your party to generate an encounter.", game_state
                
                # Generate an encounter in the current environment
                current_location = json.loads(game_state).get("current_location", "dungeon")
                environment = "dungeon"  # Default
                
                # Map location to environment type
                if "forest" in current_location.lower():
                    environment = "forest"
                elif "mountain" in current_location.lower():
                    environment = "mountain"
                elif "cave" in current_location.lower():
                    environment = "cave"
                elif "city" in current_location.lower() or "town" in current_location.lower():
                    environment = "urban"
                elif "ruin" in current_location.lower():
                    environment = "ruins"
                elif "swamp" in current_location.lower():
                    environment = "swamp"
                
                encounter_data = generate_encounter_with_narrative(
                    party=party_data,
                    difficulty=difficulty,
                    environment=environment
                )
                
                # Extract relevant info
                narrative = encounter_data.get("narrative", "")
                encounter = encounter_data.get("encounter", {})
                monsters = encounter.get("monsters", {})
                
                # Summarize monsters for DM response
                monster_summary = []
                for name, data in monsters.items():
                    count = data.get("count", 1)
                    if count == 1:
                        monster_summary.append(f"1 {name}")
                    else:
                        monster_summary.append(f"{count} {name}s")
                
                monster_text = ", ".join(monster_summary)
                
                # Update game state
                game_state_dict = json.loads(game_state)
                game_state_dict["game_history"].append(f"Encountered: {monster_text}")
                game_state_dict["current_scene"] = narrative
                
                return session_history + f"\n\nDM: {narrative}\n\n[Encounter generated: {monster_text} - {encounter.get('difficulty', 'medium')} difficulty]", json.dumps(game_state_dict)
            
            except Exception as e:
                return session_history + f"\n\nDM: I couldn't generate an encounter. Error: {str(e)}", game_state
        
        elif command == "scene":
            # Generate a scene description
            setting = "current location"
            if len(command_parts) > 1:
                setting = command_parts[1]
            
            try:
                game_state_dict = json.loads(game_state)
                party_data = game_state_dict.get("party", [])
                current_location = game_state_dict.get("current_location", "")
                
                scene_context = {
                    "setting": setting if setting != "current location" else current_location,
                    "mood": random.choice(["mysterious", "peaceful", "tense", "foreboding", "joyful"]),
                    "time": random.choice(["dawn", "morning", "midday", "afternoon", "dusk", "night", "midnight"]),
                    "special_element": random.choice([
                        "A mysterious statue with glowing eyes",
                        "An unusual arrangement of stones",
                        "Signs of a recent struggle",
                        "A abandoned camp with still-warm embers",
                        "Strange marks on the ground",
                        "A beautiful view of the surrounding landscape"
                    ])
                }
                
                scene_description = generate_narrative("scene_description", scene_context, party=party_data)
                
                # Update game state
                game_state_dict["current_scene"] = scene_description
                
                return session_history + f"\n\nDM: {scene_description}", json.dumps(game_state_dict)
            
            except Exception as e:
                return session_history + f"\n\nDM: I couldn't generate a scene description. Error: {str(e)}", game_state
        
        elif command == "npc":
            # Generate an NPC interaction
            npc_name = "stranger"
            if len(command_parts) > 1:
                npc_name = command_parts[1]
            
            try:
                game_state_dict = json.loads(game_state)
                party_data = game_state_dict.get("party", [])
                
                # Check if this NPC already exists in the campaign
                npcs_met = game_state_dict.get("npcs_met", {})
                npc_exists = npc_name in npcs_met
                
                if not npc_exists:
                    # Generate personality traits for new NPC
                    personality = random.choice([
                        "Friendly and helpful",
                        "Gruff but fair",
                        "Suspicious of strangers",
                        "Overly enthusiastic",
                        "Secretive and nervous",
                        "Proud and haughty",
                        "Wise and contemplative"
                    ])
                    
                    role = random.choice([
                        "Merchant",
                        "Guard",
                        "Scholar",
                        "Farmer",
                        "Noble",
                        "Adventurer",
                        "Craftsperson",
                        "Innkeeper",
                        "Religious figure"
                    ])
                else:
                    # Use existing NPC info
                    npc_info = npcs_met[npc_name]
                    personality = npc_info.get("description", "").split(",")[0]
                    role = npc_info.get("description", "").split(",")[1] if "," in npc_info.get("description", "") else "Local"
                
                npc_context = {
                    "npc_name": npc_name,
                    "npc_role": role,
                    "personality": personality,
                    "situation": random.choice([
                        "Providing information",
                        "Asking for help",
                        "Trading goods",
                        "Sharing a rumor",
                        "Warning about danger",
                        "Offering a quest"
                    ])
                }
                
                npc_dialogue = generate_narrative("npc_interaction", npc_context, party=party_data)
                
                # Update game state
                if not npc_exists:
                    game_state_dict.setdefault("npcs_met", {})[npc_name] = {
                        "description": f"{personality}, {role}",
                        "relationship": "neutral"
                    }
                
                game_state_dict["game_history"].append(f"Interacted with {npc_name} the {role}")
                
                return session_history + f"\n\nDM: {npc_dialogue}", json.dumps(game_state_dict)
            
            except Exception as e:
                return session_history + f"\n\nDM: I couldn't generate an NPC interaction. Error: {str(e)}", game_state
        
        elif command == "quest":
            # Generate a quest
            try:
                game_state_dict = json.loads(game_state)
                current_location = game_state_dict.get("current_location", "")
                
                # Generate a quest appropriate for the location and party
                party_data = game_state_dict.get("party", [])
                party_levels = [char.get("level", 1) for char in party_data]
                avg_level = sum(party_levels) / max(len(party_levels), 1)
                
                level_range = f"{max(1, int(avg_level))}-{max(2, int(avg_level) + 2)}"
                
                quest = generate_adventure_premise(
                    theme=random.choice([
                        "Lost artifact recovery",
                        "Missing person investigation",
                        "Monster infestation",
                        "Local mystery",
                        "Bandit trouble",
                        "Ancient ruins exploration"
                    ]),
                    level_range=level_range,
                    location=current_location,
                    villain_type=random.choice([
                        "Bandit leader",
                        "Corrupt official",
                        "Local monster",
                        "Cult leader",
                        "Rival adventurer",
                        "Ancient evil"
                    ])
                )
                
                # Extract title from the generated quest
                lines = quest.split('\n')
                title = lines[0].replace('#', '').strip() if lines else "New Quest"
                
                # Update game state with new quest
                game_state_dict.setdefault("quest_log", []).append({
                    "title": title,
                    "description": quest,
                    "status": "active"
                })
                
                game_state_dict["game_history"].append(f"Received quest: {title}")
                
                return session_history + f"\n\nDM: {quest}", json.dumps(game_state_dict)
            
            except Exception as e:
                return session_history + f"\n\nDM: I couldn't generate a quest. Error: {str(e)}", game_state

        else:
            return session_history + f"\n\nDM: Unknown command '/{command}'. Type /help for available commands.", game_state
    
    # Regular input - generate a narrative response
    try:
        game_state_dict = json.loads(game_state)
        
        # Build context from game state
        party_summary = []
        for character in game_state_dict.get("party", []):
            party_summary.append(f"{character.get('name', 'Unnamed')} ({character.get('race', '')} {character.get('class', '')})")
        
        party_text = ", ".join(party_summary) if party_summary else "No party members"
        location = game_state_dict.get("current_location", "Unknown location")
        
        # Create a context string for the LLM
        context = f"""
        Party: {party_text}
        Current Location: {location}
        Current Scene: {game_state_dict.get('current_scene', '')}
        
        Recent History:
        {'. '.join(game_state_dict.get('game_history', [])[-5:])}
        
        Active Quests:
        {', '.join([q.get('title', 'Unnamed Quest') for q in game_state_dict.get('quest_log', []) if q.get('status') == 'active'])}
        
        The player just said: "{user_input}"
        """
        
        # Generate DM response using the narrative system
        user_prompt = f"""
        As the Dungeon Master, respond to the player's input: "{user_input}"
        
        Consider the current game state and provide an engaging, descriptive response.
        If the player is asking about the world, provide vivid descriptions.
        If they want to take an action, describe the outcome.
        If they're interacting with an NPC, continue the dialogue naturally.
        
        Keep your response in-character as a DM would speak to players.
        """
        
        response = genai.GenerativeModel(
            model_name=generation_model,
            generation_config={
                "temperature": 0.7,
                "top_p": 0.95,
                "max_output_tokens": 500
            }
        ).generate_content(
            [
                {"role": "system", "parts": [system_prompt + "\n" + context]},
                {"role": "user", "parts": [user_prompt]}
            ]
        )
        
        dm_response = response.text
        
        # Add the response to session history
        session_history += f"\n\nDM: {dm_response}"
        
        # Update game state with this interaction
        game_state_dict["game_history"].append(f"Player: {user_input[:50]}{'...' if len(user_input) > 50 else ''}")
        
        return session_history, json.dumps(game_state_dict)
    
    except Exception as e:
        return session_history + f"\n\nDM: I encountered an error processing your input. Error: {str(e)}", game_state


# Function to add character to the party
def add_character_to_party(name, race, char_class, level):
    """Generate and add a character to the party"""
    if not name or not race or not char_class:
        return "Please fill in all fields", campaign.party
    
    try:
        level_val = int(level)
        if level_val < 1 or level_val > 20:
            return "Level must be between 1 and 20", campaign.party
    except:
        return "Level must be a number", campaign.party
    
    try:
        character = generate_character(
            character_class=char_class,
            character_race=race,
            level=level_val,
            name=name
        )
        
        campaign.party.append(character)
        campaign.save()
        return f"Added {name} to the party!", campaign.party
    except Exception as e:
        return f"Error generating character: {str(e)}", campaign.party

# Function to remove character from the party
def remove_character(character_index):
    """Remove a character from the party"""
    try:
        idx = int(character_index)
        if 0 <= idx < len(campaign.party):
            removed = campaign.party.pop(idx)
            campaign.save()
            return f"Removed {removed.get('name', 'character')} from the party", campaign.party
        else:
            return "Invalid character index", campaign.party
    except:
        return "Please select a character to remove", campaign.party

# Function to update location
def update_location(new_location):
    """Update the party's current location"""
    if not new_location:
        return "Please enter a location", campaign.current_location
    
    campaign.update_location(new_location)
    return f"Party is now at: {new_location}", new_location

# Create the Gradio interface
def create_dm_interface():
    """Create and launch the DM interface"""
    with gr.Blocks(title="D&D DM Assistant") as app:
        gr.Markdown("# 🐉 D&D Dungeon Master Assistant")
        
        with gr.Tab("Interactive DM Session"):
            gr.Markdown("""
            ### Welcome to your interactive D&D campaign!
            
            Talk to the DM naturally or use commands:
            - `/help` - Show available commands
            - `/roll XdY` - Roll dice (e.g., `/roll 2d6`)
            - `/rules [query]` - Look up rules
            - `/encounter [difficulty]` - Generate a random encounter
            - `/scene [setting]` - Describe a new scene
            - `/npc [name]` - Interact with an NPC
            - `/quest` - Generate a new quest
            """)
            
            with gr.Row():
                with gr.Column(scale=2):
                    chatbot = gr.Textbox(
                        value="Welcome to your D&D adventure! Type your actions or use commands like /help",
                        label="Game Session",
                        lines=20
                    )
                    
                    user_input = gr.Textbox(
                        placeholder="What do you do? (or use / commands)",
                        label="Your Action",
                        lines=2
                    )
                    
                    submit_btn = gr.Button("Submit", variant="primary")
                    
                    # Hidden state to track game state
                    game_state = gr.State(json.dumps(campaign.__dict__))
                
                with gr.Column(scale=1):
                    gr.Markdown("### Party Members")
                    party_display = gr.Dataframe(
                        headers=["Name", "Race", "Class", "Level"],
                        datatype=["str", "str", "str", "number"],
                        label="Current Party",
                        value=[[c.get("name", ""), c.get("race", ""), c.get("class", ""), c.get("level", 1)] 
                               for c in campaign.party]
                    )
                    
                    gr.Markdown("### Current Location")
                    location_display = gr.Textbox(
                        value=campaign.current_location,
                        label="Location"
                    )
                    
                    new_location = gr.Textbox(
                        placeholder="Enter new location",
                        label="Change Location"
                    )
                    update_loc_btn = gr.Button("Update Location")
                    
                    gr.Markdown("### Active Quests")
                    quest_display = gr.Dataframe(
                        headers=["Title", "Status"],
                        datatype=["str", "str"],
                        label="Quest Log",
                        value=[[q.get("title", ""), q.get("status", "")] for q in campaign.quest_log]
                    )
        
        with gr.Tab("Character Management"):
            gr.Markdown("### Character Management")
            
            with gr.Row():
                with gr.Column():
                    gr.Markdown("#### Add New Character")
                    char_name = gr.Textbox(placeholder="Character Name", label="Name")
                    char_race = gr.Dropdown(
                        choices=["Human", "Elf", "Dwarf", "Halfling", "Half-Orc", "Gnome", "Tiefling", "Dragonborn"],
                        label="Race"
                    )
                    char_class = gr.Dropdown(
                        choices=["Fighter", "Wizard", "Cleric", "Rogue", "Barbarian", "Bard", "Druid", "Monk", "Paladin", "Ranger", "Sorcerer", "Warlock"],
                        label="Class"
                    )
                    char_level = gr.Slider(minimum=1, maximum=20, value=1, step=1, label="Level")
                    add_char_btn = gr.Button("Add to Party")
                    add_result = gr.Textbox(label="Result")
                
                with gr.Column():
                    gr.Markdown("#### Current Party")
                    party_list = gr.Dataframe(
                        headers=["Index", "Name", "Race", "Class", "Level"],
                        datatype=["number", "str", "str", "str", "number"],
                        label="Party Members",
                        value=[[i, c.get("name", ""), c.get("race", ""), c.get("class", ""), c.get("level", 1)] 
                               for i, c in enumerate(campaign.party)]
                    )
                    
                    remove_idx = gr.Number(label="Character Index to Remove", precision=0)
                    remove_btn = gr.Button("Remove from Party")
                    remove_result = gr.Textbox(label="Result")
        
        with gr.Tab("Rules Reference"):
            gr.Markdown("### D&D Rules Reference")
            
            rule_query = gr.Textbox(
                placeholder="Ask a question about D&D rules",
                label="Rule Question"
            )
            rule_submit = gr.Button("Look Up Rule")
            rule_answer = gr.Markdown(label="Rule Answer")
        
        # Set up event handlers
        submit_btn.click(
            dm_session,
            inputs=[user_input, game_state, chatbot],
            outputs=[chatbot, game_state]
        )
        
        add_char_btn.click(
            add_character_to_party,
            inputs=[char_name, char_race, char_class, char_level],
            outputs=[add_result, party_list]
        )
        
        remove_btn.click(
            remove_character,
            inputs=[remove_idx],
            outputs=[remove_result, party_list]
        )
        
        update_loc_btn.click(
            update_location,
            inputs=[new_location],
            outputs=[location_display, location_display]
        )
        
        rule_submit.click(
            lambda q: query_rules(q),
            inputs=[rule_query],
            outputs=[rule_answer]
        )
        
        # Update displays when game state changes
        def update_displays(state_json):
            try:
                state = json.loads(state_json)
                party = state.get("party", [])
                party_data = [[i, c.get("name", ""), c.get("race", ""), c.get("class", ""), c.get("level", 1)] 
                             for i, c in enumerate(party)]
                
                party_simple = [[c.get("name", ""), c.get("race", ""), c.get("class", ""), c.get("level", 1)] 
                               for c in party]
                
                quest_data = [[q.get("title", ""), q.get("status", "")] 
                             for q in state.get("quest_log", [])]
                
                location = state.get("current_location", "Unknown")
                
                return party_data, party_simple, quest_data, location
            except:
                return [], [], [], "Error"
        
        game_state.change(
            update_displays,
            inputs=[game_state],
            outputs=[party_list, party_display, quest_display, location_display]
        )
    
    return app

# Create and launch the interface
interactive_dm = create_dm_interface()

# For Jupyter Notebook display
interactive_dm.launch(inline=True, share=False)

Loaded existing campaign.
* Running on local URL:  http://127.0.0.1:7860

To create a public link, set `share=True` in `launch()`.


Error generating adventure premise: 400 Content with system role is not supported.
Error generating character: 400 Content with system role is not supported.
Falling back to basic character generation...


# Evaluation and Reflection

In [35]:
import time
from collections import defaultdict

# Track performance metrics
performance_metrics = {
    "response_times": [],
    "success_rates": defaultdict(list),
    "component_usage": defaultdict(int)
}

# Function to evaluate a component with timing
def evaluate_component(component_name, function, *args, **kwargs):
    """Evaluate a component's performance and track metrics"""
    start_time = time.time()
    try:
        result = function(*args, **kwargs)
        success = True
    except Exception as e:
        result = None
        success = False
        print(f"Error in {component_name}: {e}")
    
    end_time = time.time()
    duration = end_time - start_time
    
    # Record metrics
    performance_metrics["response_times"].append(duration)
    performance_metrics["success_rates"][component_name].append(success)
    performance_metrics["component_usage"][component_name] += 1
    
    return result, duration, success

# Run evaluation on key components
print("🔍 Evaluating D&D DM Assistant Components\n")

# Test rules lookup
test_rule_queries = [
    "How does combat work?",
    "What are the special features of a Wizard?",
    "How do I calculate my character's ability modifier?",
    "What happens when a character is blinded?",
    "How does a saving throw work?"
]

print("Evaluating Rules Knowledge Base:")
for query in test_rule_queries:
    result, duration, success = evaluate_component("rules_lookup", rate_limited_query, query)
    print(f"  - Query: '{query}'")
    print(f"    Time: {duration:.2f}s, Success: {success}")

# Test character generation
print("\nEvaluating Character Generation:")
result, duration, success = evaluate_component(
    "character_generation", 
    generate_character,
    character_class="Paladin", 
    character_race="Dragonborn", 
    level=3
)
print(f"  - Generated character in {duration:.2f}s, Success: {success}")

# Test narrative generation
print("\nEvaluating Narrative Generation:")
scene_context = {
    "setting": "Mountain fortress",
    "mood": "tense",
    "time": "dusk",
    "special_element": "Massive dragon skeleton embedded in the wall"
}
result, duration, success = evaluate_component(
    "narrative_generation", 
    generate_narrative,
    "scene_description", 
    scene_context
)
print(f"  - Generated narrative in {duration:.2f}s, Success: {success}")

# Test encounter generation
print("\nEvaluating Encounter Generation:")
result, duration, success = evaluate_component(
    "encounter_generation", 
    generate_encounter_with_narrative,
    party=None, 
    difficulty="hard", 
    environment="cave"
)
print(f"  - Generated encounter in {duration:.2f}s, Success: {success}")

# Display summary metrics
print("\n📊 Performance Summary:")
avg_response_time = sum(performance_metrics["response_times"]) / len(performance_metrics["response_times"])
print(f"Average response time: {avg_response_time:.2f}s")

for component, successes in performance_metrics["success_rates"].items():
    success_rate = sum(successes) / len(successes) * 100
    print(f"{component}: {success_rate:.1f}% success rate ({performance_metrics['component_usage'][component]} calls)")

# Visual analysis (optional)
try:
    plt.figure(figsize=(10, 6))
    plt.bar(performance_metrics["component_usage"].keys(), [sum(performance_metrics["success_rates"][c])/len(performance_metrics["success_rates"][c]) for c in performance_metrics["component_usage"].keys()])
    plt.title("Component Success Rates")
    plt.ylabel("Success Rate")
    plt.ylim(0, 1)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()
except Exception as e:
    print(f"Could not generate chart: {e}")

🔍 Evaluating D&D DM Assistant Components

Evaluating Rules Knowledge Base:
  - Query: 'How does combat work?'
    Time: 1.31s, Success: True
  - Query: 'What are the special features of a Wizard?'
    Time: 0.81s, Success: True
  - Query: 'How do I calculate my character's ability modifier?'
    Time: 0.92s, Success: True
  - Query: 'What happens when a character is blinded?'
    Time: 0.98s, Success: True
  - Query: 'How does a saving throw work?'
    Time: 0.99s, Success: True

Evaluating Character Generation:
Error generating character: 400 Content with system role is not supported.
Falling back to basic character generation...
  - Generated character in 0.09s, Success: True

Evaluating Narrative Generation:
Error generating narrative: 400 Content with system role is not supported.
  - Generated narrative in 0.17s, Success: True

Evaluating Encounter Generation:
Error generating narrative: 400 Content with system role is not supported.
  - Generated encounter in 0.09s, Success: True

<Figure size 1000x600 with 1 Axes>

## Project Reflection

### System Strengths
- **Integration of Multiple AI Capabilities**: The system successfully combines multiple AI capabilities (knowledge retrieval, character generation, narrative creation, and encounter building) into a cohesive DM assistant.
- **Flexibility**: The assistant can handle various aspects of D&D gameplay, from rules questions to dynamic narrative generation.
- **Interactivity**: The Gradio interface provides an accessible way for users to interact with all aspects of the system.
- **State Persistence**: Campaign state is maintained between sessions, allowing for continuity in gameplay.

### Limitations and Challenges
- **Limited Rules Knowledge**: The system only has access to a simplified subset of D&D rules, limiting its ability to answer complex edge cases.
- **Generation Constraints**: Character, narrative, and encounter generation is constrained by predefined templates and the capabilities of the underlying LLM.
- **Response Time**: Some operations, particularly those requiring multiple API calls, can be slow, impacting the real-time feel of a tabletop session.
- **Error Handling**: While the system includes error handling, unexpected inputs or API limitations can still cause issues.
- **Limited Game Logic**: The system doesn't have true game logic and relies on narrative descriptions rather than mechanical resolution of actions.

### Ethical Considerations
- **Content Appropriateness**: The system should generate age-appropriate content for the intended audience.
- **Creative Ownership**: Consider attribution for AI-generated narratives and characters, especially if used in published works.
- **Human Creativity**: The tool should enhance human creativity rather than replace the creative aspects of the DM role.
- **Dependency on External APIs**: The system relies on external AI services, which raises questions about data privacy and service availability.

### Future Improvements
- **Expanded Rules Database**: Incorporate a more comprehensive set of D&D rules, possibly by scraping SRD content or integrating with existing rule databases.
- **Fine-tuned LLM**: Fine-tune the underlying language model specifically for D&D terminology and conventions.
- **Combat System**: Implement a more structured combat system with initiative tracking, damage calculation, and status effects.
- **Multi-Modal Capabilities**: Add image generation for characters, monsters, and scenes to enhance the visual experience.
- **Voice Interface**: Add speech-to-text and text-to-speech capabilities for a more immersive experience.
- **Community Content**: Allow users to share and import campaign elements like characters, encounters, and adventures.
- **Context-Aware Responses**: Improve the assistant's awareness of the ongoing narrative to provide more coherent and consistent responses.

### User Testing Results
For future development, formal user testing would be valuable, collecting feedback on:
- Interface usability
- Response quality and appropriateness
- Session flow and continuity
- Feature gaps and priorities

### Conclusion
This D&D DM Assistant demonstrates the potential for AI to enhance tabletop role-playing games by handling routine tasks, providing creative inspiration, and managing game state. While it cannot replace the creativity and adaptability of a human Dungeon Master, it shows promise as a tool to assist new DMs, help with session preparation, and enhance the gaming experience.

In [37]:
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.patches import Rectangle, FancyArrowPatch

# Create system architecture diagram
plt.figure(figsize=(12, 8))

# Define component positions
components = {
    "User Interface": (0.5, 0.9),
    "Campaign State": (0.5, 0.7),
    "Rules Knowledge Base": (0.2, 0.5),
    "Character Manager": (0.5, 0.5),
    "Narrative Generator": (0.8, 0.5),
    "Encounter Builder": (0.5, 0.3),
    "LLM (Gemini)": (0.5, 0.1)
}

# Draw components
for name, (x, y) in components.items():
    plt.annotate(name, xy=(x, y), xycoords='axes fraction', 
                 bbox=dict(boxstyle="round,pad=0.5", fc="lightblue", ec="blue", alpha=0.8),
                 ha='center', va='center', fontsize=12)

# Draw arrows
arrows = [
    ("User Interface", "Campaign State"),
    ("Campaign State", "Rules Knowledge Base"),
    ("Campaign State", "Character Manager"),
    ("Campaign State", "Narrative Generator"),
    ("Campaign State", "Encounter Builder"),
    ("Rules Knowledge Base", "LLM (Gemini)"),
    ("Character Manager", "LLM (Gemini)"),
    ("Narrative Generator", "LLM (Gemini)"),
    ("Encounter Builder", "LLM (Gemini)")
]

for start, end in arrows:
    start_x, start_y = components[start]
    end_x, end_y = components[end]
    plt.annotate("", xy=(end_x, end_y), xytext=(start_x, start_y),
                arrowprops=dict(arrowstyle="->", color="gray", lw=1.5),
                xycoords='axes fraction', textcoords='axes fraction')

plt.title("D&D DM Assistant System Architecture", fontsize=16)
plt.axis('off')
plt.tight_layout()
plt.show()

<Figure size 1200x800 with 1 Axes>

## Competitive Analysis

| Feature | This Project | D&D Beyond | Roll20 | Existing AI DMs |
|---------|-------------|------------|--------|-----------------|
| Rules Knowledge | Limited SRD | Complete | Complete | Varies |
| Character Generation | AI-based | Form-based | Form-based | Limited |
| Narrative Creation | Dynamic LLM | None | Limited | Basic |
| Campaign Management | Basic | Limited | Advanced | Minimal |
| Encounter Building | Difficulty-aware | Limited | Manual | Basic |
| User Interface | Simple Gradio | Web-based | Complex | Command-line |
| Cost | Free/API costs | Subscription | Freemium | Varies |

### Unique Value Proposition
This project combines the flexibility of AI-generated content with the structure and rules of D&D in a way that commercial tools don't currently offer. While platforms like D&D Beyond and Roll20 provide excellent reference material and game management, they lack the creative assistant capabilities this system provides through generative AI.

## User Testing Methodology

### Testing Plan
1. **Participant Selection**: Recruit 5-10 D&D players with varying experience levels (new players, experienced players, DMs)
2. **Testing Sessions**: Conduct 30-45 minute guided sessions with specific tasks:
   - Look up a D&D rule
   - Generate and modify a character
   - Create a narrative scene
   - Build an encounter
   - Run a small interactive session

### Feedback Collection
- **Task Success Metrics**: Time to completion, error rate
- **Satisfaction Surveys**: System Usability Scale (SUS) questionnaire
- **Open-ended Questions**:
  - What was most helpful about the assistant?
  - What features were most confusing or difficult to use?
  - How would you integrate this into your actual D&D sessions?
  - What additional features would you want to see?

### Sample Survey Questions
1. I think that I would like to use this system frequently.
2. I found the system unnecessarily complex.
3. The generated content was high quality and useful for D&D.
4. I would need technical support to use this system.
5. The various functions in this system were well integrated.

### Iteration Plan
After gathering feedback, prioritize improvements based on:
1. Critical usability issues that prevent core functionality
2. Features that received the most positive feedback (to strengthen)
3. Most requested missing features
4. Performance optimizations

In [43]:
with open("requirements.txt", "w") as f:
    f.write("""
google-generativeai>=0.3.0
langchain>=0.1.0
langchain-community>=0.0.10
langchain-google-genai>=0.0.5
faiss-cpu>=1.7.4
python-dotenv>=1.0.0
gradio>=4.0.0
pandas>=2.0.0
numpy>=1.24.0
matplotlib>=3.7.0
beautifulsoup4>=4.12.0
requests>=2.31.0
""")

print("✅ Created requirements.txt file for deployment")

deployment_instructions = """
## Deployment Options

### Local Deployment
1. Clone the repository
2. Install requirements: `pip install -r requirements.txt`
3. Set up environment variables in a `.env` file with your API keys:
    ```
    GOOGLE_API_KEY=your_google_api_key
    ```
4. Run the application: `python app.py`

### Cloud Deployment (Hugging Face Spaces)
1. Fork this repository to your GitHub account
2. Create a new Space on Hugging Face Spaces
3. Link your GitHub repository
4. Add your API key as a secret in the Space settings
5. Configure the Space to install the requirements and run app.py

### Docker Deployment
1. Build the Docker image: `docker build -t dnd-dm-assistant .`
2. Run the container: `docker run -p 7860:7860 -e GOOGLE_API_KEY=your_api_key_here dnd-dm-assistant`

### Resource Requirements
- Minimum 4GB RAM recommended
- Storage: ~500MB for base application
- API costs: Varies based on usage (~$0.001-0.01 per request)
"""

print(deployment_instructions)

✅ Created requirements.txt file for deployment

## Deployment Options

### Local Deployment
1. Clone the repository
2. Install requirements: `pip install -r requirements.txt`
3. Set up environment variables in a `.env` file with your API keys:
    ```
    GOOGLE_API_KEY=your_google_api_key
    ```
4. Run the application: `python app.py`

### Cloud Deployment (Hugging Face Spaces)
1. Fork this repository to your GitHub account
2. Create a new Space on Hugging Face Spaces
3. Link your GitHub repository
4. Add your API key as a secret in the Space settings
5. Configure the Space to install the requirements and run app.py

### Docker Deployment
1. Build the Docker image: `docker build -t dnd-dm-assistant .`
2. Run the container: `docker run -p 7860:7860 -e GOOGLE_API_KEY=your_api_key_here dnd-dm-assistant`

### Resource Requirements
- Minimum 4GB RAM recommended
- Storage: ~500MB for base application
- API costs: Varies based on usage (~$0.001-0.01 per request)



## Data Collection and Privacy Policy

### Data Collection
This application collects the following data:
- User inputs in the chat interface
- Generated character data
- Campaign state information
- Query patterns for rules lookups

### Data Usage
Collected data is used for:
- Maintaining session state
- Improving system responses
- Debugging and error analysis

### Data Sharing
- API queries are sent to external services (Google Gemini)
- Google's privacy policy applies to data processed by their APIs
- Campaign data remains local unless explicitly shared

### User Controls
- All locally stored data can be deleted by removing the data/ directory
- Users can export their campaign data and character sheets
- No personal identification information is required to use the system

### Compliance
- The system uses SRD content which is freely available
- Generated content should be reviewed for compliance with D&D copyright
- Commercial use would require additional licensing considerations